In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

import os.path
import nltk

In [ ]:
from settings import *
from utils import *

In [ ]:
def argumentExtraction(limitDocuments=10):
    query = createQuery(limitDocuments, 'web')
    documents = queryDB(query)
    t, b = documents.iat[3,0], documents.iat[3,1]

    sent_text = nltk.sent_tokenize(b)
    return sent_text
    
    #loadGloveEmbeddings(gloveFile)
    #print(t)
    #print(b)    
    #print (documents['title'].to_string())
argumentExtraction()    